# Title

## Introduction

Minecraft is a sandbox video game where players explore and build in a world made of blocks. In this world of infinite possibilities, we would like to analyze the relationship between player behaviors and their demographics. This data was collected by a Computer Science research group at UBC led by Frank Wood. The method in which they collected data was recording players' actions throughout the world in their Minecraft servers. They provide two data sets, one of player data that contains a list of unique players and their data, and another that contains a list of individual game sessions made by each player. Specifically, we would like to know what Minecraft player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how each player differs from each other.
### Question:
Can the variables played_hours, number of sessions played ~gender, experience,~ and age predict whether or not a Minecraft player will subscribe to a game-related newsletter?

As we are only looking at player demographic and behavioural data, we will only use the first data set, players.csv that contains the list of players and their respective game data. This data set contains 196 total observations and 7 variables: subscribe, gender, age, name, played_hours, hashedEmail and experience. Out of all the variables, the predictors of subscribing are experience, played_hours, gender and age. The columns email and name will be left out as these varibles do not contribute to newsletter subscription.

***(version for using both datasets if everyone is okay with it) ***
We will be using both datasets to explore this question. The players.csv data contains the list of players, 196, and seven variables for each player. The variables are subscribe, gender, age, name, played_hours, hashedEmail, and experience. To predict wether a player has subscribed we will use the quantitative variables of age and played_hours. We will also use sessions.csv which records each session played for a total of 1535 observations. The sessions data includes the same hashedEmail identifier as players.csv, start time and end time, as well as original start and end time. The only information we will take from sessions.csv is the number of occurrences of each hashedEmail which can also be interpretted as the number of sessions a player has played.

- The subscribe variable is categorical and contains 2 possible values: "TRUE" or "FALSE". This shows whether a player has subscribed to a game related newsletter or not.

~- The experience variable is categorical and contains 5 possible values: "Beginner", "Amateur", "Regular", "Pro" and "Veteran". These placeholders describe the level of skill and experience each player has rated themselves from least to most experienced.~

- The played_hours variable is numeric and up to 1 decimal place. This variable, quantified in hours, was determined through how long each player played Minecraft in one sitting.

~- The gender variable is categorical and contains 7 options: "Male", "Female", "Non-binary", "Prefer not to say", "Agender", "Two-spirited" and "Other." This variable shows the gender of the player.~

- The age variable is numeric and contains integer values. This variable is the age of the player.

A noticeable issue is that this data contains some outliers in the played_hours variable. Some players have claimed to play over 200 hours of Minecraft in a single session. These points may affect the classification model in predictions. 



## Methods & Results

We began by wrangling and cleaning the dataset. Specifically, we converted any “Prefer not to say” entries in the gender column into NA values and changed the column name “Age” to “age” so that all variable names follow a lowercase format. This ensures that functions using arguments like na.rm = TRUE will work properly during our analysis.

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(cowplot)
options(repr.matrix.max.rows = 6) # i got these loading steps from other assignments we've done. im not really sure
#source('cleanup.R') # if they are all necesary tho so feel free to remove

# loading in dataset
players = read_csv("data/players.csv")
players

# changing age column name + converting "Prefer not to say" into NAs
clean_data <- players |>
  mutate(age = Age,
    gender = na_if(gender, "Prefer not to say")   # convert to NA
  ) |>
  select(-Age)
clean_data

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


experience,subscribe,hashedEmail,played_hours,name,gender,age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,NA,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


(If we want to make it a bit more complicated) First we need to load in both dataset and wrangle them seperately. For the session.csv data we will group the 1535 observations by the hashedEmail to find the number of sessions each individual has played For players.csv we will change the variable name 'Age' to 'age' to follow a consistent naming pattern, change the subscribe variable to a factor datatype, and deselect the variables we do not need such as name, experience, and gender. From here we can merge the two datasets and deselect the hashedEmail as it will no longer be needed. 

In [8]:
sessions = read_csv("data/sessions.csv")

# combining the observations so that each hashedEmail has the number of sessions played beside it
indiv_sessions <- sessions |> 
group_by(hashedEmail) |>
summarize(sessions_played = n()) 

#data wrangeling for players
predictor_data <- players |>
    mutate(age = Age,
          subscribe = as.factor(subscribe)) |>
    select(-experience, -name, -gender) |>
    drop_na()

complete_data <- merge(x = predictor_data, y = indiv_sessions, by = "hashedEmail")
minecraft_data <- select(complete_data, -hashedEmail)
minecraft_data

Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


subscribe,played_hours,Age,age,sessions_played
<fct>,<dbl>,<dbl>,<dbl>,<int>
TRUE,1.5,20,20,2
FALSE,0.4,21,21,1
TRUE,0.1,17,17,1
⋮,⋮,⋮,⋮,⋮
TRUE,56.1,23,23,310
TRUE,0.1,17,17,1
TRUE,1.1,20,20,1


We have to do a summary of the cleaned data here:

"performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis

creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis"

Now we can start our analysis by tuning and training our model. We will spilt the data into 75% training and 25% testing. Now using minecraft_training we can create a recipe and find the most accurate value of K. We did a  five fold cross validation and tested for values ranging from 1-20.

In [3]:
set.seed(1243)

minecraft_split <- initial_split(minecraft_data, prop = 0.75, strata = subscribe)  
minecraft_train <- training(minecraft_split)   
minecraft_test <- testing(minecraft_split)

minecraft_recipe <- recipe(subscribe ~., data = minecraft_train) |>
    step_center(all_predictors()) |>
    step_scale(all_predictors())

knn_tune <- nearest_neighbor(weight_func = 'rectangular' , neighbors = tune()) |>
      set_engine('kknn') |>
      set_mode('classification')

k_vals <- tibble(neighbors = seq(from = 1, to = 20, by = 1))

minecraft_vfold <- vfold_cv(minecraft_train, v = 5, strata = subscribe)

knn_results <- workflow() |>
      add_recipe(minecraft_recipe) |>
      add_model(knn_tune) |>
      tune_grid(resamples = minecraft_vfold, grid = k_vals) |>
      collect_metrics()
knn_results


neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
1,accuracy,binary,0.6058136,5,0.03590804,Preprocessor1_Model01
1,roc_auc,binary,0.5607692,5,0.05013555,Preprocessor1_Model01
2,accuracy,binary,0.6058136,5,0.03590804,Preprocessor1_Model02
⋮,⋮,⋮,⋮,⋮,⋮,⋮
19,roc_auc,binary,0.6284615,5,0.021749776,Preprocessor1_Model19
20,accuracy,binary,0.7394909,5,0.006910322,Preprocessor1_Model20
20,roc_auc,binary,0.5672527,5,0.037497977,Preprocessor1_Model20


We can visualize the accuracy of each K on the plot below and use the slice_max function to find that the best value for K is 11.

neighbors,.metric,.estimator,mean,n,std_err,.config
<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
11,accuracy,binary,0.7611971,5,0.02739855,Preprocessor1_Model11


In [9]:
accuracies <- knn_results |> 
      filter(.metric == 'accuracy') 

accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
      geom_point() +
      geom_line() +
      labs(x = "Neighbors", y = "Accuracy Estimate") +
      ggtitle('Plot -: Number of K vs Accuracy')
accuracy_versus_k 

best_k <- slice_max(accuracies, order_by= mean, n=1)
best_k

Knowing that our ideal value of K is 11 we can retrain the model using the same recipe as before.

In [12]:
knn <- nearest_neighbor(weight_func = 'rectangular' , neighbors = 11) |>
      set_engine('kknn') |>
      set_mode('classification')

knn_fit <- workflow() |>
      add_recipe(minecraft_recipe) |>
      add_model(knn) |>
      fit(data = minecraft_train)

Now having the fully trained model we can predict the values of the testing set.

In [14]:
test_predictions <- predict(knn_fit, minecraft_test) |>
bind_cols(minecraft_test)
test_predictions
# mnist_metrics <- mnist_predictions |>
#     metrics(truth = y, estimate = .pred_class)  |>
#     filter(.metric == "accuracy")

.pred_class,age,played_hours,subscribe,sessions_played
<fct>,<dbl>,<dbl>,<fct>,<int>
TRUE,21,0.7,TRUE,1
TRUE,19,0.6,TRUE,2
TRUE,17,1.2,TRUE,2
⋮,⋮,⋮,⋮,⋮
TRUE,17,3.8,TRUE,3
TRUE,21,3.5,TRUE,4
TRUE,20,1.1,TRUE,1
